In [30]:
import scipy as sp
import numpy as np
from skbio import DistanceMatrix,DNA,TreeNode 


def hamming_distance(sequence1, sequence2):
    n, m = len(sequence1), len(sequence2)
    return sum(sequence1[i] != sequence2[i] for i in range(min(n, m))) + abs(n-m)
def example_guide_tree_generation(sequences, metric=hamming_distance):
    guide_dm = DistanceMatrix.from_iterable(sequences, metric=metric, key='id')
    print(guide_dm)
    print(type(guide_dm))
    print(guide_dm.condensed_form())
    guide_lm = sp.cluster.hierarchy.average(guide_dm.condensed_form())
    print(guide_lm)
    print(type(guide_lm))
    guide_tree = TreeNode.from_linkage_matrix(guide_lm, guide_dm.ids)
    return guide_tree

# not the best thing out there 

In [31]:
query_of_sequences = [DNA("ACGCGATGACCGGGCCTTGTAAAAT", metadata={'id': 'seq1'}),
                        DNA("ATGATGACAGGGCTTGTAACT", metadata={'id': 'seq2'}),
                        DNA("TTCATGACCGGCTTATACTTAT", metadata={'id': 'seq3'}),
                        DNA("ACCCTACCTGTCGTATTGTAAT", metadata={'id': 'seq4'}),
                      ]
result =example_guide_tree_generation(query_of_sequences)
result

4x4 distance matrix
IDs:
'seq1', 'seq2', 'seq3', 'seq4'
Data:
[[ 0. 25. 25. 25.]
 [25.  0. 22. 22.]
 [25. 22.  0. 22.]
 [25. 22. 22.  0.]]
<class 'skbio.stats.distance._base.DistanceMatrix'>
[25. 25. 25. 22. 22. 22.]
[[ 1.  2. 22.  2.]
 [ 3.  4. 22.  3.]
 [ 0.  5. 25.  4.]]
<class 'numpy.ndarray'>


<TreeNode, name: unnamed, internal node count: 2, tips count: 4>